In [1]:
import pandas as pd
from patternly.detection import AnomalyDetection
from sklearn.cluster import KMeans

original = pd.read_csv("./01_2015_LANL-01A_SOPA_MPA.txt", low_memory=False)

In [2]:
predictions = pd.DataFrame()
window_size = 1000
for j in range(19):
    data = original.iloc[:,j+1]
    seqs = pd.concat(
        [data[(window_size * i):(window_size * (i + 1))].reset_index(drop=True) for i in range(int(data.shape[0]/1000))], 
        axis=1
    ).T.reset_index(drop=True)

    seqs

    kmeans = KMeans(n_clusters=1)
    pipeline = AnomalyDetection(
        anomaly_sensitivity=1, 
        clustering_alg=kmeans,
        quantize_type="simple",
        eps=0.05,
        verbose=False,
    )
    pipeline.fit(seqs)

    predictions = pd.concat([predictions, pd.DataFrame(pipeline.predict())], axis=1)
#     anomalies = predictions[predictions[0] == True]
#     anomalies

predictions.columns = range(predictions.shape[1])

In [3]:
import numpy as np
sums = predictions.apply(np.sum, axis=1).to_frame()
sums[sums[0] == predictions.shape[1]]
sums[sums[0] > predictions.shape[1] / 2]

,0
42,15
100,11
150,19
201,14
231,10
